In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re

2024-02-11 11:10:07.339976: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-11 11:10:07.617911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-11 11:10:07.622585: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-11 11:10:09.001643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [53]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/yuvraj/nltk_data...


True

In [2]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [3]:
(train, test), info = tfds.load(
    'ag_news_subset',  # Specify the dataset name
    split=['train', 'test'],  # Load both training and testing sets
    as_supervised=True,  # Load as (text, label) pairs
    with_info=True  # Include dataset metadata
)

2024-02-11 11:10:21.128509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-11 11:10:21.262795: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
print(info)

tfds.core.DatasetInfo(
    name='ag_news_subset',
    full_name='ag_news_subset/1.0.0',
    description="""
    AG is a collection of more than 1 million news articles. News articles have been
    gathered from more than 2000 news sources by ComeToMyHead in more than 1 year of
    activity. ComeToMyHead is an academic news search engine which has been running
    since July, 2004. The dataset is provided by the academic comunity for research
    purposes in data mining (clustering, classification, etc), information retrieval
    (ranking, search, etc), xml, data compression, data streaming, and any other
    non-commercial activity. For more information, please refer to the link
    http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html .
    
    The AG's news topic classification dataset is constructed by Xiang Zhang
    (xiang.zhang@nyu.edu) from the dataset above. It is used as a text
    classification benchmark in the following paper: Xiang Zhang, Junbo Zhao, Yann
    LeCu

In [5]:
train_df = tfds.as_dataframe(train, info)
test_df=tfds.as_dataframe(test,info)

In [7]:
train_df=pd.DataFrame(train_df)
test_df=pd.DataFrame(test_df)

In [8]:
train_df.rename(columns={'description':'text'},inplace=True)
test_df.rename(columns={'description':'text'},inplace=True)

In [9]:
train_df

,text,label
0,b'AMD #39;s new dual-core Opteron chip is desi...,3
1,b'Reuters - Major League Baseball\\Monday anno...,1
2,b'President Bush #39;s quot;revenue-neutral q...,2
3,b'Britain will run out of leading scientists u...,3
4,"b'London, England (Sports Network) - England m...",1
...,...,...
119995,b'Ivan Ljubicic edged No. 7 seed Joachim Johan...,1
119996,b'MANAMA: A \\$1.3 billion (BD491 million) com...,2
119997,"b'And lo, the hawk begat the dove. At least th...",0
119998,"b'West Palm Beach, FL (Sports Network) - Tom L...",1


In [10]:
test_df

,text,label
0,"b' CHARLOTTE, N.C. (Sports Network) - Carolina...",1
1,"b' PRESENTACION, Philippines (Reuters) - Phili...",0
2,"b'SAN FRANCISCO, CA - November 30, 2004 -Ubiso...",3
3,b'DANIEL VETTORI spun New Zealand to the brink...,1
4,"b""AFP - Georgian and South Ossetian forces ove...",0
...,...,...
7595,b'It took 2 days for Russia #39;s security ser...,0
7596,b'MONTREAL : Air Canada said it sealed a deal ...,2
7597,b'The administration issued a report indicatin...,3
7598,"b'LOS GATOS, Calf. Shares of mail-order DVD re...",2


In [11]:
print(train_df['label'].unique())
print(test_df['label'].unique())

[3 1 2 0]
[1 0 3 2]


In [34]:
def clean_text(text):
    text=str(text)
    tokens = word_tokenize(text)
    tokens[0]=tokens[0][2:]
    cleaned_text=[lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stopwords and word not in string.punctuation]
    return ' '.join(cleaned_text)

In [36]:
train_df['text']=train_df['text'].apply(clean_text)
test_df['text']=test_df['text'].apply(clean_text)

In [37]:
train_df

,text,label
0,amd 39 new dual-core opteron chip designed mai...,3
1,reuters major league baseball\\monday announce...,1
2,president bush 39 quot revenue-neutral quot ta...,2
3,britain run leading scientist unless science e...,3
4,london england sport network england midfielde...,1
...,...,...
119995,ivan ljubicic edged 7 seed joachim johansson t...,1
119996,manama \\ 1.3 billion bd491 million combined p...,2
119997,lo hawk begat dove least one imago private eye...,0
119998,west palm beach fl sport network tom lehman na...,1


In [38]:
test_df

,text,label
0,charlotte n.c. sport network carolina panther ...,1
1,presentacion philippine reuters philippine com...,0
2,san francisco ca november 30 2004 -ubisoft one...,3
3,daniel vettori spun new zealand brink crushing...,1
4,`` afp georgian south ossetian force overnight...,0
...,...,...
7595,took 2 day russia 39 security service announce...,0
7596,montreal air canada said sealed deal brazil 39...,2
7597,administration issued report indicating emissi...,3
7598,los gatos calf share mail-order dvd rental com...,2


In [16]:
train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv',index=False)

In [190]:
X_train= tf.data.Dataset.from_tensor_slices(train_df['text'].values)
Y_train=tf.data.Dataset.from_tensor_slices(train_df['label'].values)
X_test= tf.data.Dataset.from_tensor_slices(test_df['text'].values)
Y_test=tf.data.Dataset.from_tensor_slices(test_df['label'].values)

In [171]:
for text in X_train.take(1):
    print(text)
for label in Y_train.take(1):
    print(label)

tf.Tensor(b"b'amd 39 new dual-core opteron chip designed mainly corporate computing application including database web service financial transaction", shape=(), dtype=string)
tf.Tensor(3, shape=(), dtype=int64)


In [191]:
for text in X_test.take(1):
    print(text)
for label in Y_test.take(1):
    print(label)

tf.Tensor(b'b charlotte n.c. sport network carolina panther running back stephen davis miss remainder season placed injured reserve saturday', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)
